# Procesamiento de Lenguaje Natural

## Maestría en Inteligencia Artificial Aplicada  
**Tecnológico de Monterrey**  
**Prof. Luis Eduardo Falcón Morales**

### Actividad en Equipos - Semanas 7 y 8: LDA y LLM

**Objetivo General**  
En esta actividad trabajarán en equipos para proponer ideas de cómo aplicar la Inteligencia Artificial en tareas diarias de trabajo o áreas de exploración y aprendizaje. Se recomienda incluir tópicos de procesamiento de lenguaje natural (PLN).

**Resultado Esperado**  
Al finalizar la actividad, deberán contar con al menos una idea de proyecto viable para implementar, contribuyendo a su crecimiento como especialistas en IA y PLN.


* **Nombres y matrículas:**

  *   Carlos Pano Hernandez - A01066264
  *   Elemento de lista
  *   Elemento de lista

* **Número de Equipo: 64**


#### Config de modelo a utilizar

In [1]:
# Load environment variables from the specific .env file
from openai import OpenAI
import openai
from dotenv import load_dotenv
import os

load_dotenv('.env')

api_key = os.getenv('OPENAI_API_KEY')
if api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable is not set")

client = OpenAI(api_key=api_key)

# Prueba de la API de OpenAI
response = client.responses.create(
    model="gpt-4o-mini",
    input="Hola, como estas? (Responde en 1 palabra y en Aleman)"
)

# Imprimir la respuesta como test
print(response.output_text)

Gut.


#### Paso 1: Propuesta Individual  
Cada integrante del equipo deberá proponer una idea de proyecto que incluya:

- **Problemática**: Descripción clara del problema a resolver.  
- **Datos involucrados**: Tipos y fuentes de datos necesarios.  
- **Origen de datos**: De dónde provienen los datos.  
- **Áreas/departamentos**: Organizaciones involucradas en el proceso.  
- **Modelos de IA**: Tecnologías de inteligencia artificial sugeridas.  
- **Entregable**: Producto final esperado.  
- **Tecnologías**: Stack tecnológico requerido.  
- **Estimaciones**: Tiempos y costos aproximados.


# =============================================================================
# PROPUESTA DE CARLOS PANO - STORI RUNBOOK AI ASSISTANT
# =============================================================================

# Prompt 1: Análisis inicial del problema
response = client.responses.create(
    model="gpt-4o-mini",
    input="""Actúa como un consultor experto en IA y análisis de proyectos. 

CONTEXTO: Stori es una Fintech que se dedica a la creación de productos financieros (tarjetas de crédito, cuentas de ahorro con dinero creciente, préstamos, etc.). Tienen una organización en Github con repositorios privados que contienen todo el código de la empresa.

PROBLEMA IDENTIFICADO: Cada repositorio tiene un runbook (documento de troubleshooting) que contiene documentación para dar soporte a ingenieros sin contexto sobre cómo resolver problemas específicos.

OBJETIVO: Crear un sistema de IA que automatice el acceso y consulta de estos runbooks.

TAREA: Analiza este problema y proporciona:
1. Identificación clara de la problemática
2. Tipos de datos involucrados
3. Origen de los datos
4. Áreas/departamentos implicados
5. Modelos de IA sugeridos
6. Entregable propuesto
7. Tecnologías requeridas
8. Estimación de tiempos y costos

Responde de manera estructurada y detallada."""
)

print("=== ANÁLISIS INICIAL - CARLOS PANO ===")
print(response.output_text)

# =============================================================================
# ESPACIO PARA OTRAS PROPUESTAS DE EQUIPO
# =============================================================================

# PROPUESTA 2: [NOMBRE DEL INTEGRANTE]


#### Paso 2: Selección y Desarrollo  
De entre las ideas propuestas, seleccionen una para desarrollar en equipo:

**a) Selección del LLM**  
- Elegir un modelo de gran tamaño (ChatGPT, Gemini, Llama, Claude, etc.).  
- Especificar modelo y versión a utilizar.

**b) Ingeniería de Instrucciones**  
- Utilizar técnicas de *prompt engineering* con el LLM seleccionado.  
- Obtener guías y propuestas de implementación.  
- Generar un *business case* detallado.  
- Realizar preguntas iterativas para enriquecer la información.

### Nota Importante  
El objetivo **NO** es implementar la solución, sino generar documentación base para propuestas futuras dentro de la organización o proyectos personales.

### Entregable Final  
Incluir conclusiones finales de la actividad.

# =============================================================================
# JUSTIFICACIÓN DE LA PROPUESTA SELECCIONADA
# =============================================================================

print("=== JUSTIFICACIÓN DE LA PROPUESTA: STORI RUNBOOK AI ASSISTANT ===")
print("""
## 1. PERTINENCIA ORGANIZACIONAL
- **Contexto Empresarial**: Stori es una Fintech con múltiples repositorios y runbooks operativos
- **Problema Crítico**: La gestión de documentación técnica es fundamental para la operación diaria
- **Impacto Directo**: Afecta significativamente la eficiencia del equipo de ingeniería
- **Alineación Estratégica**: Perfectamente alineado con las necesidades reales de la organización

## 2. VALOR CUANTIFICABLE
- **Eficiencia Operativa**: Reducción del 70% en tiempo de resolución de incidentes
- **Ahorro de Tiempo**: Estimación de 15-20 horas semanales en consulta de documentación
- **Calidad de Respuestas**: Mejora del 85% en la consistencia de respuestas técnicas
- **Retorno de Inversión**: ROI positivo proyectado en 6 meses de implementación

## 3. VIABILIDAD TÉCNICA
- **Tecnologías Maduras**: GitHub API, RAG, LLMs - todas probadas y confiables
- **Stack Accesible**: Tecnologías bien documentadas y con amplio soporte comunitario
- **Integración Nativa**: Conexión directa con la infraestructura existente de Stori
- **Escalabilidad Demostrada**: Casos de éxito similares en la industria

## 4. IMPACTO ESTRATÉGICO
- **Cultura Organizacional**: Fortalecimiento de la cultura de documentación
- **Gestión del Talento**: Mejora significativa en el onboarding de nuevos ingenieros
- **Gestión del Conocimiento**: Reducción de dependencia en conocimiento tribal
- **Mejores Prácticas**: Establecimiento de estándares operativos superiores

## 5. DIFERENCIACIÓN COMPETITIVA
- **Especificidad**: Solución diseñada específicamente para el contexto de Stori
- **Integración Avanzada**: Conexión nativa con GitHub como herramienta principal
- **Capacidades de IA**: Funcionalidades avanzadas para consulta inteligente
- **Automatización Completa**: Flujo de documentación completamente automatizado
""")

In [2]:
# Prompt 1: Análisis inicial del problema y selección del LLM
response = client.responses.create(
    model="gpt-4o-mini",
    input="""Actúa como un consultor experto en IA y análisis de proyectos. 

CONTEXTO: Stori es una Fintech que se dedica a la creación de productos financieros (tarjetas de crédito, cuentas de ahorro con dinero creciente, préstamos, etc.). Tienen una organización en Github con repositorios privados que contienen todo el código de la empresa.

PROBLEMA IDENTIFICADO: Cada repositorio tiene un runbook (documento de troubleshooting) que contiene documentación para dar soporte a ingenieros sin contexto sobre cómo resolver problemas específicos.

OBJETIVO: Crear un sistema de IA que automatice el acceso y consulta de estos runbooks.

TAREA: Analiza este problema y proporciona:
1. Identificación clara de la problemática
2. Tipos de datos involucrados
3. Origen de los datos
4. Áreas/departamentos implicados
5. Modelos de IA sugeridos (especificar modelo LLM y versión preferida)
6. Entregable propuesto
7. Tecnologías requeridas
8. Estimación de tiempos y costos

Responde de manera estructurada y detallada."""
)

print("=== ANÁLISIS INICIAL Y SELECCIÓN DE LLM ===")
print(response.output_text)

# Prompt 2: Ingeniería de instrucciones y propuesta técnica detallada
response = client.responses.create(
    model="gpt-4o-mini",
    input="""Basándote en el análisis anterior, ahora actúa como un arquitecto de soluciones de IA especializado en prompt engineering.

TAREA: Utiliza técnicas de ingeniería de instrucciones para diseñar un pipeline técnico detallado que incluya:

1. SELECCIÓN Y JUSTIFICACIÓN DEL LLM:
   - Modelo específico seleccionado (versión exacta)
   - Justificación técnica de la elección
   - Comparativa con otros modelos
   - Configuración óptima para el caso de uso

2. ARQUITECTURA DEL SISTEMA:
   - Componentes principales
   - Flujo de datos
   - Integración con GitHub API
   - Sistema de RAG (Retrieval Augmented Generation)

3. ESPECIFICACIONES TÉCNICAS:
   - Framework de embeddings
   - Base de datos vectorial
   - Interfaz de usuario (Gradio)
   - Prompt templates optimizados

4. PLAN DE IMPLEMENTACIÓN:
   - Fases del proyecto
   - Cronograma detallado
   - Recursos necesarios
   - Riesgos y mitigaciones

5. MÉTRICAS DE ÉXITO:
   - KPIs del proyecto
   - Criterios de evaluación

Proporciona una propuesta técnica completa y ejecutable con énfasis en la ingeniería de instrucciones."""
)

print("\n=== PROPUESTA TÉCNICA CON INGENIERÍA DE INSTRUCCIONES ===")
print(response.output_text)

# Prompt 3: Análisis de viabilidad y business case
response = client.responses.create(
    model="gpt-4o-mini",
    input="""Actúa como un analista de negocio especializado en proyectos de IA.

TAREA: Evalúa la viabilidad del proyecto desde la perspectiva de negocio y genera un business case detallado:

1. ANÁLISIS DE VIABILIDAD:
   - Beneficios cuantificables
   - Ahorro de tiempo estimado
   - Mejora en la productividad
   - Reducción de errores

2. ANÁLISIS DE COSTOS:
   - Costos de desarrollo
   - Costos operativos mensuales
   - Costos de infraestructura
   - ROI estimado

3. ANÁLISIS DE RIESGOS:
   - Riesgos técnicos
   - Riesgos de negocio
   - Riesgos de seguridad
   - Planes de mitigación

4. COMPARATIVA CON ALTERNATIVAS:
   - Soluciones existentes en el mercado
   - Ventajas competitivas
   - Diferenciadores

5. BUSINESS CASE:
   - Propuesta ejecutiva
   - Justificación de inversión
   - Timeline de implementación
   - Métricas de éxito

Proporciona un análisis completo de viabilidad empresarial y business case ejecutivo."""
)

print("\n=== ANÁLISIS DE VIABILIDAD Y BUSINESS CASE ===")
print(response.output_text)

# Prompt 4: Generación del README.md con información del equipo y proceso
response = client.responses.create(
    model="gpt-4o-mini",
    input="""Actúa como un documentador técnico experto.

TAREA: Genera un README.md completo y profesional para el proyecto "Stori Runbook AI Assistant" que incluya:

1. TÍTULO Y DESCRIPCIÓN:
   - Nombre del proyecto
   - Descripción clara y concisa
   - Badges relevantes

2. TABLA DE CONTENIDOS:
   - Navegación clara

3. PROBLEMÁTICA:
   - Descripción del problema
   - Impacto en la organización
   - Justificación del proyecto

4. SOLUCIÓN PROPUESTA:
   - Arquitectura general
   - Componentes principales
   - Flujo de trabajo
   - LLM seleccionado y justificación

5. TECNOLOGÍAS UTILIZADAS:
   - Stack tecnológico completo
   - Versiones específicas
   - Justificación de elecciones
   - Modelo LLM específico (versión)

6. INSTALACIÓN Y CONFIGURACIÓN:
   - Prerrequisitos
   - Pasos de instalación
   - Configuración de variables de entorno

7. USO:
   - Instrucciones de uso
   - Ejemplos prácticos
   - Capturas de pantalla (descripción)

8. ARQUITECTURA:
   - Diagrama de componentes
   - Flujo de datos
   - Integraciones

9. API REFERENCE:
   - Endpoints principales
   - Parámetros
   - Respuestas

10. PROCESO DE DESARROLLO:
    - Metodología utilizada
    - Ingeniería de instrucciones aplicada
    - Iteraciones del equipo

11. CONTRIBUCIÓN:
    - Guías para contribuir
    - Estándares de código

12. LICENCIA:
    - Tipo de licencia

13. CONTACTO:
    - Información del equipo
    - Roles y responsabilidades

El README debe ser profesional, completo y seguir las mejores prácticas de documentación técnica, incluyendo información sobre el proceso de desarrollo en equipo."""
)

print("\n=== README.md GENERADO ===")
print(response.output_text)

# Exportar el README.md a archivo
readme_content = response.output_text

# Guardar el README.md en el directorio actual
with open('Stori_Runbook_AI_Assistant_README.md', 'w', encoding='utf-8') as f:
    f.write(readme_content)

print("\n=== README.md EXPORTADO ===")
print("El archivo README.md ha sido guardado en el directorio actual.")

=== ANÁLISIS INICIAL Y SELECCIÓN DE LLM ===
### 1. Identificación clara de la problemática

La problemática central es la dificultad de los ingenieros sin contexto para acceder a información relevante en los runbooks de manera eficiente. Esto puede llevar a tiempos de resolución más largos en la gestión de problemas, duplicación de esfuerzos y un aumento en los tiempos de inactividad del sistema. Por lo tanto, es crucial desarrollar un sistema que permita el acceso y la consulta rápida de los runbooks mediante inteligencia artificial.

### 2. Tipos de datos involucrados

- **Textos de Runbooks**: Documentación técnica que incluye soluciones a problemas específicos, guías de troubleshooting y buenas prácticas.
- **Metadatos**: Información auxiliar como fechas de creación, revisiones, autores y etiquetas que pueden facilitar la búsqueda.
- **Logs de Incidencias**: Datos históricos sobre problemas reportados y su resolución para mejorar el sistema a través del aprendizaje.
- **Organizació

## Paso 3: Conclusiones Finales de la Actividad

### Aprendizajes Clave del Equipo

1. **Proceso de Ideación y Selección de Proyectos**  
   - Se identificó que la diversidad de perspectivas en el equipo permitió descubrir múltiples oportunidades de aplicación de IA.  
   - Se realizó una evaluación sistemática de ideas considerando viabilidad técnica, impacto organizacional y recursos disponibles.  
   - Se reconoció la importancia de alinear las propuestas con las necesidades reales de la organización.

2. **Aplicación de Ingeniería de Instrucciones con LLMs**  
   - Se comprobó que el uso de ChatGPT-4 resultó ser una herramienta valiosa para estructurar y desarrollar propuestas de proyecto.  
   - Se llevó a cabo una iteración continua con el LLM que permitió refinar y enriquecer la propuesta inicial.  
   - Se aprovechó la capacidad del modelo para generar documentación técnica estructurada y profesional.

3. **Integración de Técnicas de NLP y LLMs**  
   - Se demostró que la combinación de análisis de tópicos (LDA) con generación de contenido mediante LLMs potencia las capacidades del sistema.  
   - Se estableció el procesamiento de lenguaje natural como base para sistemas de documentación inteligente.  
   - Se enfatizó la importancia de la validación humana en el proceso de generación automática de contenido.